In [44]:
# import the libraries

import pandas as pd
import numpy as np
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [14]:
#import data set

df1 = pd.read_csv("archive(1)/combined_data_1.txt")
df2 = pd.read_csv("archive(1)/combined_data_2.txt")
df3 = pd.read_csv("archive(1)/combined_data_3.txt")
df4 = pd.read_csv("archive(1)/combined_data_4.txt")

In [8]:
df = pd.read_csv('Churn_Modelling.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [51]:
# split the data into training validation and test set
train,val,test = np.split(df.sample(frac=1,random_state=42),[int(.6*len(df)),int(.8*(len(df)))])
x_train = (train.iloc[:,:13]).float()
y_train = (train.iloc[:,13:]).float()
trainer = TensorDataset(x_train,y_train)
Trainloader = DataLoader(trainer,batch_size = 32,shuffle = True, drop_last = True)


x_val = t.Tensor(val.iloc[:,:13])
y_val = t.Tensor(val.iloc[:,13:])
validator = TensorDataset(x_val,y_val)
valloader = DataLoader(validator,batchsize = validator.tensors[0].shape[0])

x_test = t.Tensor(test.iloc[:,:13])
y_test = t.Tensor(test.iloc[:,13:])
tester = TensorDataset(x_test, y_test)
testloader = DataLoader(tester, batchsize = tester.tensors[0].shape[0])

AttributeError: 'DataFrame' object has no attribute 'float'

In [38]:
# set the device to use gpu if available
t.device = ('CUDA' if t.cuda.is_available() else 'CPU')

In [41]:
# design the neural network

def classifynet():
    
    class churnpredNET(nn.modules):
        
        def __init__(self):
            super.__init__()
            
            # input layer
            self.input = nn.Linear(13,100)
            
            # hidden/fully connected
            self.fc1 = nn.Linear(100,100)
            self.fc2 = nn.Linear(100,100)
            
            # output layer
            self.output = nn.Linear(100,2)
            
        def forward(self,x):
            x = F.relu(self.input(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc1(x))
            return t.log_softmax(self.output(x),axis = 1)
        
    # model instance
    net = churnpredNET()

    # loss function
    loss = nn.NLLLoss()

    # optimiser
    optimizer = t.optim.SGD(net.parameters(),lr = .1)
    
    return net,loss, optimizer
   

In [ ]:
# test the model with one batch
net,lossfun,optimizer = classifynet()

X,y = iter(train_loader).next()
yHat = net(X)

# values are log-probability of each number (0-9)
# print(torch.exp(yHat))

# now let's compute the loss
loss = lossfun(yHat,y)
print(' ')
print('Loss:')
print(loss)